# Building and analyzing EMMAA Models of Neurofibromatosis

## 1. A Knowledge Network of Molecular Mechanisms from the Neurofibromatosis Literature

In this section we put together an initial neurofibromatosis-specific knowledge network based on molecular interactions we have compiled from reading all available abstracts and full text articles from PubMed.

As background, we use multiple text mining systems from our collaborators and from the research community to extract raw causal relations and then assemble them using our knowledge assembly system, INDRA (the Integrated Network and Dynamical Reasoning Assembler). We do this systematically across all available literature and store the results in the INDRA Database for use in downstream projects like this one. The INDRA Database is publicly accessible at https://db.indra.bio. 

The first step in creating the EMMAA model for Neurofibromatosis is to find publications broadly relevant for neurofibromatosis. We do this by querying based on the both the term "neurofibromatosis" as well as the top-level Medical Subject Heading (MESH) term for Neurofibromatoses, D017253 (see https://www.ncbi.nlm.nih.gov/mesh/68017253). We use the `indra.literature` module to search PubMed using the PubMed REST API.

In [34]:
# Preliminaries
import json
from processor import NlModelProcessor
from indra.tools import assemble_corpus as ac
from indra.assemblers.pysb import PysbAssembler
from indra.explanation.model_checker.pysb import PysbModelChecker
from indra.literature import pubmed_client
from indra.pipeline import AssemblyPipeline

In [27]:
# Query PubMed for articles related to NF
def get_nf_pmids():
    # PMIDs from text search for "neurofibromatosis"
    nf_text_pmids = pubmed_client.get_ids('neurofibromatosis')
    # Mesh term for neurofibromatoses
    nf_mesh_ids = pubmed_client.get_ids_for_mesh('D017253')
    return list(set(nf_text_pmids + nf_mesh_ids))

# Get the PMIDs
nf_pmids = get_nf_pmids()

Here's how many unique PMIDs we found:

In [28]:
len(nf_pmids)

17882

We then query the INDRA Database to get the INDRA Statements that have been extracted from these papers by text mining. One complication here is that for the EMMAA model we need to get raw, unassembled statements from the INDRA Database, which are currently not available from the public APIs (the public APIs provide access to the "finished", assembled statements). Therefore for this example we show the code that we run internally to obtain these statements but, by default, load the statements from a cache.

In [32]:
# Get raw statement IDs from the DB for the given TextRefs
reload = False
pmid_stmts_file = 'nf_raw_stmts.pkl'
if reload: 
    db = get_primary_db()
    # Get statements for the given text refs
    print(f"Distilling statements for {len(nf_pmids)} TextRefs")
    start = time.time()
    clauses = [
        db.TextRef.pmid.in_(nf_pmids),
        db.TextContent.text_ref_id == db.TextRef.id,
        db.Reading.text_content_id == db.TextContent.id,
        db.RawStatements.reading_id == db.Reading.id]
    pmid_stmts = distill_stmts(db, get_full_stmts=True, clauses=clauses)
    end = time.time()
    elapsed = end - start
    print(elapsed)
    list(ac.dump_statements(pmid_stmts, pmid_stmts_file))
else:
    pmid_stmts = list(ac.load_statements(pmid_stmts_file))

INFO: [2020-11-11 17:47:12] indra.tools.assemble_corpus - Loading nf_raw_stmts.pkl...
INFO: [2020-11-11 17:47:13] indra.tools.assemble_corpus - Loaded 20209 statements


Let's see how many statements we got:

In [33]:
len(pmid_stmts)

20209

### Preprocessing, filtering and deduplicating statements

The statements that we've obtained from the INDRA Database in the previous step represent raw output from NLP systems. We need to preprocess these statements in several ways: we need to fix named entity normalization, standardize sites of posttranslation modifications to canonical protein sequences, identify duplicate statements and overlapping statements, etc. The steps of this procedure are described in the `config.json` file for the EMMAA model. The EMMAA system reads the `config.json` file and runs assembly on the raw statements in AWS Batch. For illustration purposes we do this ourselves here.

In [37]:
# Load the config file
with open('../nf_model/config.json', 'rt') as f:
    nf_config = json.load(f)

Here are the assembly steps for the model:

In [36]:
nf_config['assembly']

[{'function': 'filter_no_hypothesis'},
 {'function': 'ground_statements',
  'kwargs': {'mode': 'local', 'ungrounded_only': True}},
 {'function': 'ground_statements',
  'kwargs': {'mode': 'local',
   'sources': ['sparser'],
   'ungrounded_only': False}},
 {'function': 'map_grounding'},
 {'function': 'filter_grounded_only'},
 {'function': 'map_sequence'},
 {'function': 'filter_eidos_ungrounded'},
 {'function': 'run_preassembly', 'kwargs': {'return_toplevel': False}},
 {'function': 'filter_human_only'},
 {'function': 'filter_by_curation',
  'args': [{'function': 'get_curations'},
   'any',
   ['correct', 'act_vs_amt', 'hypothesis']],
  'kwargs': {'update_belief': True}},
 {'function': 'filter_top_level'}]

In [38]:
ap = AssemblyPipeline(nf_config['assembly'])

In [39]:
pmid_stmts_asmb = ap.run(pmid_stmts)

INFO: [2020-11-11 17:58:12] indra.pipeline.pipeline - Running the pipeline
INFO: [2020-11-11 17:58:12] indra.pipeline.pipeline - Calling filter_no_hypothesis
INFO: [2020-11-11 17:58:12] indra.tools.assemble_corpus - Filtering 20209 statements to no hypothesis...
INFO: [2020-11-11 17:58:12] indra.tools.assemble_corpus - 19962 statements after filter...
INFO: [2020-11-11 17:58:12] indra.pipeline.pipeline - Calling ground_statements


ModuleNotFoundError: No module named 'nltk'

## 2. A Mechanistic Ras-NF Model from Natural Language

Next we show how a second, distinct Ras-NF mechanistic model can be automatically assembled from a set of manually curated sentences describing molecular interactions. We call this approach "natural language modeling" and it is described in detail in our paper:

Gyori B.M.\*, Bachman J.A.\*, Subramanian K., Muhlich J.L., Galescu L., Sorger P.K. From word models to executable models of signaling networks using automated assembly (2017), Molecular Systems Biology, 13, 954. (https://www.embopress.org/doi/full/10.15252/msb.20177651)


### Getting the statements from text

In this step we process the assertions in the text file `ras_nf_model.rst` into INDRA Statements. Each sentence is processed individually and the statements for each sentence are stored in a local cache. Processing can take several minutes.

In [9]:
model_text_file = 'ras_nf_model.rst'
stmt_output_file = 'ras_nf_stmts.pkl'

nlmp = NlModelProcessor(model_text_file, cache_dir='cache', trips_endpoint='drum-dev')
ras_stmts = nlmp.get_statements(stmt_output_file)

Processing group 'EGFR Pathway'...
1 of 13: processing 'EGF binds to EGFR.'...


INFO: [2020-11-11 16:15:12] processor - Saving 1 stmts in EGFbindstoEGFR.pkl


2 of 13: processing 'EGFR bound to EGF binds EGFR bound to EGF.'...


INFO: [2020-11-11 16:15:15] processor - Saving 1 stmts in EGFRboundtoEGFbindsEGFRboundtoEGF.pkl


3 of 13: processing 'EGFR bound to EGFR phosphorylates itself at Y1016.'...


...

INFO: [2020-11-11 16:21:40] processor - Saving 1 stmts in ActiveTIAM1activatesRAC1.pkl


Processing group 'Miscellaneous'...
1 of 2: processing 'Unphosphorylated PEBP1 binds RAF1.'...


INFO: [2020-11-11 16:21:41] processor - Saving 1 stmts in UnphosphorylatedPEBP1bindsRAF1.pkl


2 of 2: processing 'RAF1 not bound to PEBP1 is active.'...


INFO: [2020-11-11 16:21:43] processor - Saving 1 stmts in RAF1notboundtoPEBP1isactive.pkl
INFO: [2020-11-11 16:21:43] indra.tools.assemble_corpus - Dumping 210 statements into ras_nf_stmts.pkl...


Let's see how many statements we have:

In [10]:
len(ras_stmts)

210

The NLP system we're using for this, called TRIPS (developed by the Institute for Human and Machine Cognition, IHMC), sometimes grounds the text "NF1" as a cell line instead of as the gene/protein. So we fix that here using the INDRA `map_grounding` function.

In [15]:
# Grounding map the statements
grounding_map = {'NF1': {'HGNC': '7765'}}
ras_stmts = ac.map_grounding(ras_stmts, do_rename=True, grounding_map=grounding_map)

INFO: [2020-11-11 16:28:00] indra.preassembler.grounding_mapper.disambiguate - Adeft will not be available for grounding disambiguation.
INFO: [2020-11-11 16:28:00] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
INFO: [2020-11-11 16:28:00] indra.tools.assemble_corpus - Mapping grounding on 210 statements...
INFO: [2020-11-11 16:28:00] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
INFO: [2020-11-11 16:28:00] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/johnbachman/.indra/bio_ontology/1.4/bio_ontology.pkl
INFO: [2020-11-11 16:28:07] indra.preassembler.grounding_mapper.mapper - 0 statements filtered out


To deploy the model in EMMAA, we would then run the following script:

    `python scripts/emmaa_model_from_statements.py -

## Assembling an executable PySB model

To explain the claims that we extracted from the text mined NF model, we take our detailed natural language model and assemble it into a rule-based executable model using INDRA and PySB. For more on PySB see our previous paper:

Lopez C.F.\*, Muhlich J.L.\*, Bachman J.A.\*, Sorger P.K. Programming biological models in Python using PySB (2013), Molecular Systems Biology, 9, 646. (https://www.embopress.org/doi/full/10.1038/msb.2013.1)

In [19]:
ras_stmts = stmts

In [21]:
pa = PysbAssembler(ras_stmts)
pysb_model = pa.make_model()

INFO: [2020-11-11 16:46:54] indra.assemblers.pysb.assembler - Using default assembly policy.


Getting parents for ('HGNC', '20249')
Getting parents for ('HGNC', '7765')
Getting parents for ('HGNC', '11714')
Getting parents for ('HGNC', '16262')
Getting parents for ('HGNC', '8975')
Getting parents for ('HGNC', '6407')
Getting parents for ('HGNC', '8975')
Getting parents for ('HGNC', '6125')
Getting parents for ('HGNC', '8816')
Getting parents for ('HGNC', '8993')
Getting parents for ('HGNC', '12363')
Getting parents for ('HGNC', '12362')
Getting parents for ('HGNC', '12362')
Getting parents for ('HGNC', '12363')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '28426')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '10011')
Getting parents for ('HGNC', '6769')
Getting parents for ('HGNC', '6770')
Getting parents for ('HGNC', '9829')
Getting parents for ('HGNC', '8630')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '30287')
Getting parents for ('HGNC', '3942')
Getting parents for ('HGNC', '30287')
Getting parents for ('HGNC'

## Checking the NF literature findings against our detailed model

Next we use the model checker component within INDRA to see how many of findings from our text mined NF model can be explained with the detailed mechanisms we've explicitly curated in our natural language model.

In [24]:
pmc = PysbModelChecker(pysb_model, nf_stmts[0:50])

NameError: name 'nf_stmts' is not defined

# New heading